In [17]:
# Standalone diffusion Panel app (outside -> inside) in ONE cell.
# Computes impurity diffusion profiles in a silver liner for various alloys.
# Interactive controls: alloy, thickness, temperature, time, normalization.
# Four synchronized Bokeh plots (depth/time, linear/log) with legend click-to-hide.
# Simplified (removed element filtering & autoscale to restore stable widget reactivity).
# Refactored to watcher pattern (@pn.depends(..., watch=True)) so widgets display like the other notebook.

import numpy as np
import pandas as pd
from scipy.special import erf
import panel as pn
from bokeh.plotting import figure
from bokeh.models import HoverTool
from bokeh.palettes import Category10

pn.extension()

# ------------------------------------------------------------------
# Diffusion / materials data
# ------------------------------------------------------------------
alloy_imp_dict = {
    "Ni":{
        "Ni4": dict(D0=3.1e-6, H=110.1, unit="(kJ/mol)/RT", source="Fisher.2014", comment="grain boundary"),
        "Ni5": dict(D0=2.8, H=230.4, unit="(kJ/mol)/RT", source="Mehrer.1990", comment="polycrystalline")
    },
    "Fe":{
        "Fe2": dict(D0=2.2, H=50, unit="(kcal/mol)/RT", source="Fisher.2014", comment="single crystal"),
        "Fe3": dict(D0=1.9, H=49.36, unit="(kcal/mol)/RT", source="Fisher.2014", comment="single crystal"),
        "Fe6": dict(D0=2.42, H=205.3, unit="(kJ/mol)/RT", source="Klotsmann.1990", comment="single crystal")
    },
    "Cr":{
        "Cr2": dict(D0=1.1, H=2, unit="eV/kT", source="Fisher.2014", comment="single crystal"),
        "Cr3": dict(D0=3.26, H=210, unit="(kJ/mol)/RT", source="Makuta.1979", comment="polycrystalline"),
    },
    "Co":{
        "Co1": dict(D0=1.9, H=48.75, unit="(kcal/mol)/RT", source="Fisher.2014", comment="single crystal"),
        "Co3": dict(D0=1.9, H=204.1, unit="(kJ/mol)/RT", source="Mehrer.1990", comment="single crystal"),
    },
    "Ti":{
        "Ti3": dict(D0=1.33, H=198, unit="(kJ/mol)/RT", source="Makuta.1979", comment="polycrystalline"),
    },
    "Al":{
        "Al1": dict(D0=0.13, H=159.5, unit="(kJ/mol)/RT", source="Mehrer.1990", comment="polycrystalline"),
    },
    "Mn":{
        "Mn2": dict(D0=4.29, H=196, unit="(kJ/mol)/RT", source="Makuta.1979", comment="polycrystalline"),
    },
    "Si":{
        "Si2": dict(D0=8.4e-2, H=36.5, unit="(kcal/mol)/RT", source="Parditka.2013", comment="polycrystalline"),
        "Si3": dict(D0=0.2, H=38, unit="(kcal/mol)/RT", source="Tobin.1960", comment="single crystal"),
        "Si4": dict(D0=0.084, H=152.8, unit="(kJ/mol)/RT", source="Neumann.2009", comment="polycrystalline"),
    },
    "Cu":{
        "Cu1": dict(D0=2.9e-2, H=39, unit="(kcal/mol)/RT", source="Fisher.2014", comment="single crystal"),
        "Cu5": dict(D0=1.23, H=193, unit="(kJ/mol)/RT", source="Mehrer.1990", comment="single crystal"),
        "Cu6": dict(D0=0.029, H=164, unit="(kJ/mol)/RT", source="Mehrer.1990", comment="single crystal"),
        "Cu7": dict(D0=0.029, H=164.1, unit="(kJ/mol)/RT", source="Dorner.1980", comment="single crystal"),
        "Cu8": dict(D0=0.63, H=46.2, unit="(kcal/mol)/RT", source="Tobin.1960", comment="single crystal"),
    },
    "S":{
        "S1": dict(D0=7e-1, H=38, unit="(kcal/mol)/RT", source="Fisher.2014", comment="polycrystalline"),
        "S2": dict(D0=1.65, H=167.5, unit="(kJ/mol)/RT", source="Mehrer.1990", comment="polycrystalline"),
        "S3": dict(D0=1.65, H=1.735, unit="eV/kT", source="Neumann.2009", comment="polycrystalline"),
        "S4": dict(D0=0.7, H=159.1, unit="(kJ/mol)/RT", source="Neumann.2009", comment="polycrystalline"),
    }
}

atom_database = {
    "Ni": dict(mol_mass=58.6934, dens=8.908),
    "Fe": dict(mol_mass=55.845, dens=7.874),
    "Cr": dict(mol_mass=51.9961, dens=7.15),
    "Nb": dict(mol_mass=92.906, dens=8.57),
    "Mo": dict(mol_mass=95.95, dens=10.28),
    "Co": dict(mol_mass=58.9332, dens=8.9),
    "Ti": dict(mol_mass=47.867, dens=4.5),
    "Al": dict(mol_mass=26.982, dens=2.7),
    "Mn": dict(mol_mass=54.938, dens=7.26),
    "Si": dict(mol_mass=28.086, dens=2.329),
    "Cu": dict(mol_mass=63.546, dens=8.96),
    "C": dict(mol_mass=12.001, dens=2.267),
    "Ta": dict(mol_mass=180.948, dens=16.69),
    "P": dict(mol_mass=30.9738, dens=1.82),
    "S": dict(mol_mass=32.065, dens=2.07),
    "B": dict(mol_mass=10.81, dens=2.08),
    "W": dict(mol_mass=183.84, dens=19.3),
    "Hf": dict(mol_mass=178.49, dens=13.31),
    "Zr": dict(mol_mass=91.224, dens=6.49),
}

alloys = {
    "Inconel718": dict(dens=8.26, comp={"Ni": 52.5, "Fe":19, "Cr": 19, "Nb":5.125, "Mo":3.050, "Co":1, "Ti":0.9, "Al":0.5, "Mn":0.35, "Si":0.35, "Cu":0.3, "C":0.08, "Ta":0.05, "P":0.015, "S":0.015, "B":0.006, "W":0, "Hf":0, "Zr":0}),
    "Inconel625": dict(dens=8.44, comp={"Ni": 58.6, "Fe":5, "Cr": 22, "Nb":1.75, "Mo":9, "Co":1, "Ti":0, "Al":0.4, "Mn":0.5, "Si":0, "Cu":0, "C":0, "Ta":1.75, "P":0, "S":0, "B":0, "W":0, "Hf":0, "Zr":0}),
    "Rene41": dict(dens=8.249, comp={"Ni": 49.2585, "Fe":5, "Cr": 19, "Nb":0, "Mo":9.75, "Co":11, "Ti":3.15, "Al":1.6, "Mn":0.1, "Si":0.5, "Cu":0.5, "C":0.12, "Ta":0, "P":0, "S":0.015, "B":0.0065, "W":0, "Hf":0, "Zr":0}),
    "Haynes282": dict(dens=8.28, comp={"Ni": 57, "Fe":1.5, "Cr": 20, "Nb":0, "Mo":8.5, "Co":10, "Ti":2.1, "Al":1.5, "Mn":0.3, "Si":0.15, "Cu":0, "C":0.06, "Ta":0, "P":0, "S":0, "B":0.005, "W":0, "Hf":0, "Zr":0}),
    "U720Li": dict(dens=8.09, comp={"Ni": 57.061, "Fe":0, "Cr": 16.5, "Nb":0, "Mo":3, "Co":14.5, "Ti":5.075, "Al":2.55, "Mn":0, "Si":0, "Cu":0, "C":0.09, "Ta":0, "P":0, "S":0, "B":0.015, "W":1.25, "Hf":0, "Zr":0.04}),
    "AD730": dict(dens=8.23, comp={"Ni": 58.1975, "Fe":4.3, "Cr": 16, "Nb":1.1, "Mo":3, "Co":9, "Ti":3.6, "Al":2.25, "Mn":0, "Si":0, "Cu":0, "C":0.01, "Ta":0, "P":0, "S":0, "B":0.0125, "W":2.5, "Hf":0, "Zr":0.03}),
    "CoWAlloy1": dict(dens=8.83, comp={"Ni": 32, "Fe":0, "Cr": 12, "Nb":0, "Mo":0, "Co":42.3, "Ti":2.5, "Al":6, "Mn":0, "Si":0.4, "Cu":0, "C":0.08, "Ta":1.5, "P":0, "S":0, "B":0.08, "W":3, "Hf":0.1, "Zr":0.01}),
    "CoWAlloy2": dict(dens=8.9, comp={"Ni": 32, "Fe":0, "Cr": 12, "Nb":0, "Mo":0, "Co":40.8, "Ti":0.3, "Al":9, "Mn":0, "Si":0.4, "Cu":0, "C":0.08, "Ta":0.2, "P":0, "S":0, "B":0.08, "W":5, "Hf":0.1, "Zr":0.01}),
    "TZM": dict(dens=10.22, comp={"Ni": 0, "Fe":0, "Cr": 0.12, "Nb":0, "Mo":99.3714, "Co":0.408, "Ti":0.003, "Al":0.09, "Mn":0, "Si":0.004, "Cu":0, "C":0.0008, "Ta":0.0002, "P":0, "S":0, "B":0.0008, "W":0, "Hf":0, "Zr":0})
}

ELEMENTS = list(alloy_imp_dict.keys())
PALETTE = Category10[10]

# ------------------------------------------------------------------
# Diffusion coefficient calculation helpers
# ------------------------------------------------------------------
_coeff_cache = {}

def _arrhenius(variant: dict, temp_c: float) -> float:
    T = temp_c + 273.15
    unit = variant["unit"]
    if unit == "(kJ/mol)/RT":
        const = 8.3144598e-3
    elif unit == "(kJ/mol)/kT":
        const = 1.380649e-26
    elif unit == "(kcal/mol)/RT":
        const = 8.3144598 / 4184
    elif unit == "eV/kT":
        const = 8.617333e-5
    elif unit == "/T":
        const = 1.0
    else:
        raise ValueError("Unknown unit")
    return float(variant["D0"] * np.exp(-variant["H"] / (const * T)))

def diffusion_coefficients(temp_c: float) -> np.ndarray:
    if temp_c in _coeff_cache:
        return _coeff_cache[temp_c]
    eff = []
    for element in ELEMENTS:
        variants = alloy_imp_dict[element]
        vals = []
        for var in variants.values():
            D = _arrhenius(var, temp_c)
            if var.get("comment") == "single crystal":
                D *= 1.5
            vals.append(D)
        eff.append(1.5 * np.mean(vals))  # conservative factor
    arr = np.array(eff)
    _coeff_cache[temp_c] = arr
    return arr

# ------------------------------------------------------------------
# Surface concentrations
# ------------------------------------------------------------------
AVOGADRO = 6.022e23

def alloy_surface_concentrations(alloy_key: str) -> np.ndarray:
    data = alloys[alloy_key]
    dens = data["dens"]
    comp = data["comp"]
    out = []
    for el in ELEMENTS:
        wt_frac = comp.get(el, 0.0) / 100.0
        if wt_frac == 0:
            out.append(0.0)
        else:
            mol_mass = atom_database[el]["mol_mass"]
            out.append(dens * wt_frac * AVOGADRO / mol_mass)
    return np.array(out)

_alloy_conc_df = pd.DataFrame({k: alloy_surface_concentrations(k) for k in alloys}, index=ELEMENTS)

# ------------------------------------------------------------------
# Grids
# ------------------------------------------------------------------
X_DEPTH = np.linspace(0, 0.3, 3001)          # cm
T_TIME_DAYS = np.linspace(0, 1000, 10001)    # days
T_TIME_SEC = T_TIME_DAYS * 86400.0

# ------------------------------------------------------------------
# Widgets
# ------------------------------------------------------------------
alloy_w = pn.widgets.Select(name="Alloy", options=list(alloys.keys()), value=list(alloys.keys())[0])
thickness_w = pn.widgets.FloatSlider(name="Thickness / cm", start=0.01, end=0.3, step=0.01, value=0.10)
# IntInput for snappier updates
time_w = pn.widgets.IntInput(name="Time / days", start=1, end=1000, step=10, value=10)
temperature_w = pn.widgets.IntSlider(name="Temperature / °C", start=400, end=800, step=20, value=600)
normalize_w = pn.widgets.Checkbox(name="Normalize")

# ------------------------------------------------------------------
# Figures
# ------------------------------------------------------------------
TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
fig_depth_lin = figure(width=900, height=480, x_range=(0, thickness_w.value), tools=TOOLS, title="Depth profile (linear)", x_axis_label="Thickness / cm", y_axis_label="Conc / cm^-3")
fig_depth_log = figure(width=900, height=480, x_range=(0, thickness_w.value), y_axis_type="log", tools=TOOLS, title="Depth profile (log)", x_axis_label="Thickness / cm", y_axis_label="Conc / cm^-3")
fig_time_lin = figure(width=900, height=480, x_range=(1, time_w.value), tools=TOOLS, title="Time profile (linear)", x_axis_label="Time / days", y_axis_label="Conc / cm^-3")
fig_time_log = figure(width=900, height=480, x_range=(1, time_w.value), y_axis_type="log", tools=TOOLS, title="Time profile (log)", x_axis_label="Time / days", y_axis_label="Conc / cm^-3")

for f in (fig_depth_lin, fig_depth_log, fig_time_lin, fig_time_log):
    f.add_tools(HoverTool(tooltips=[("Element", "$name"), ("x", "@x{0.000}"), ("Conc", "@y{0.00e}" )]))

lines_depth_lin = {}
lines_depth_log = {}
lines_time_lin = {}
lines_time_log = {}
for i, el in enumerate(ELEMENTS):
    color = Category10[10][i]
    lines_depth_lin[el] = fig_depth_lin.line([], [], line_width=2, color=color, name=el, legend_label=el)
    lines_depth_log[el] = fig_depth_log.line([], [], line_width=2, color=color, name=el, legend_label=el)
    lines_time_lin[el] = fig_time_lin.line([], [], line_width=2, color=color, name=el, legend_label=el)
    lines_time_log[el] = fig_time_log.line([], [], line_width=2, color=color, name=el, legend_label=el)

for f in (fig_depth_lin, fig_depth_log, fig_time_lin, fig_time_log):
    if f.legend:
        f.legend.click_policy = "hide"

status_md = pn.pane.Markdown("", sizing_mode="stretch_width")

# ------------------------------------------------------------------
# Computations
# ------------------------------------------------------------------

def compute_profiles(thickness_cm: float, time_days: int, temp_c: int, alloy_key: str, normalize: bool):
    D = diffusion_coefficients(temp_c)            # (N,)
    C0 = alloy_surface_concentrations(alloy_key)  # (N,)
    if normalize:
        C = np.where(C0 > 0, 1.0, 0.0)
    else:
        C = C0
    t_sec = max(1.0, time_days * 86400.0)
    depth_profiles = {}
    for i, el in enumerate(ELEMENTS):
        if C[i] == 0:
            depth_profiles[el] = np.zeros_like(X_DEPTH)
        else:
            arg = X_DEPTH / (2.0 * np.sqrt(D[i] * t_sec))
            depth_profiles[el] = C[i] * (1 - erf(arg))
    time_profiles = {}
    thickness = thickness_cm
    t_arr = T_TIME_SEC
    for i, el in enumerate(ELEMENTS):
        if C[i] == 0:
            y = np.zeros_like(T_TIME_DAYS)
        else:
            denom = np.sqrt(D[i] * np.where(t_arr == 0, 1e-30, t_arr))
            arg = thickness / (2.0 * denom)
            y = C[i] * (1 - erf(arg))
            y[0] = 0.0
        time_profiles[el] = y
    return depth_profiles, time_profiles, C0

# ------------------------------------------------------------------
# Layout (persistent) + watcher update
# ------------------------------------------------------------------

tabs = pn.Tabs(("Depth (lin)", fig_depth_lin), ("Depth (log)", fig_depth_log), ("Time (lin)", fig_time_lin), ("Time (log)", fig_time_log))
panel = pn.Row(tabs, pn.Column(alloy_w, thickness_w, temperature_w, time_w, normalize_w, status_md))

@pn.depends(thickness_w, time_w, temperature_w, alloy_w, normalize_w, watch=True)
def update(*events):
    thickness = thickness_w.value
    time_days = time_w.value
    temp_c = temperature_w.value
    alloy_key = alloy_w.value
    normalize = normalize_w.value
    depth_profiles, time_profiles, C0 = compute_profiles(thickness, time_days, temp_c, alloy_key, normalize)
    # ranges
    fig_depth_lin.x_range.end = thickness
    fig_depth_log.x_range.end = thickness
    fig_time_lin.x_range.end = max(1, time_days)
    fig_time_log.x_range.end = max(1, time_days)
    # push data
    for el in ELEMENTS:
        lines_depth_lin[el].data_source.data = dict(x=X_DEPTH, y=depth_profiles[el])
        lines_depth_log[el].data_source.data = dict(x=X_DEPTH, y=np.clip(depth_profiles[el], 1e-30, None))
        lines_time_lin[el].data_source.data = dict(x=T_TIME_DAYS, y=time_profiles[el])
        lines_time_log[el].data_source.data = dict(x=T_TIME_DAYS, y=np.clip(time_profiles[el], 1e-30, None))
    # status markdown
    x_idx = min(int(round(thickness / 0.0001)), len(X_DEPTH)-1)
    surface_lines = []
    for el in ELEMENTS:
        surf = depth_profiles[el][0]
        interior = depth_profiles[el][x_idx]
        if normalize:
            surf_fmt = f"{surf*100:.1f}%"; interior_fmt = f"{interior*100:.1f}%"
        else:
            surf_fmt = f"{surf:.2e}"; interior_fmt = f"{interior:.2e}" 
        surface_lines.append(f"{el}: {surf_fmt} | {interior_fmt}")
    # Force single line breaks (two spaces + newline) without blank lines
    lines_block = "".join(f"{ln}  \n" for ln in surface_lines)
    status_md.object = (f"### Scenario\n**Alloy:** {alloy_key} | **Temp:** {temp_c} °C | **Time:** {time_days} d | **Thickness:** {thickness:.3f} cm\n\n" 
                        f"Surface (x=0) | Inside (x=thickness)  \n"  # two spaces before newline for single line break
                        f"{lines_block}")

# initial populate
update()

panel

BokehModel(combine_events=True, render_bundle={'docs_json': {'7c56fa41-0180-4db1-a567-798faaa649e9': {'version…